# PROJECT - *My Way* of seeing music covers
#### Pierre-Antoine Desplaces, Anaïs Ladoy, Lou Richard

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from io import StringIO
import sys
import requests
from bs4 import BeautifulSoup
import pickle
import os
import glob
from pathlib import Path

## Notebook plan
1. Data importation
2. Clique organisation (Multi-level indexing)
3. Addition of the language and the year of each track (SHS website web-scraping)
4. Addition of the tempo and song hotness of each track (Access to track files through the cluster)
5. Determine artist location for spatial analysis
6. Addition of the genre for each track (Use of LastFM dataset and external website for genre listing)

### 1.Data importation
Download available additional files containing metadata about our dataset from the cluster (dataset/million-songs_untar/)
- tracks_per_year.txt
- unique_tracks.txt
- unique_artists.txt
- artist_location.txt

Use the Second Hand Songs (SHS) dataset that was created through a collaboration between the Million Songs team and the Second Hand Songs website (https://secondhandsongs.com/). These data are splitted into two datasets to allowed machine learnings algorithms (a train and a test set).
- SHS_testset.txt
- SHS_trainset.txt

The use of external dataset (LastFM) for the genres and the use of the track files (.h5) available through the cluster are commented in part 4 and 5.

- All the additional files were downloaded from the cluster giving all the metadata of the Million Songs dataset. They will help to elaborate a plan and a script will then search more information about a specific track (h5 files in the cluster) maybe using cluster cpu. The path to access to a track in the cluster is for example million-songs/data/A/A/A (with the 3 letters at the end being the 3rd, 4th and 5th letter on the track id).
- The music covers will be detected using another dataset (SecondHandSongs), we have the choice to use the downloadable dataset containing 18,196 tracks (all with a connection to the MSD dataset), or to web-scrapp the SHS website (https://secondhandsongs.com/) where we have much more information (522 436 covers) but not necessarly connected to our MSD dataset. The SHS API is RESTful (return a JSON object) and we are limited to 100 requests per minute and 1000 requestion per hour but we can contact them to remove limitation.
- Some artist are geolocalised (30% of the MSD total artists) on the artist_location dataframe.

In [ ]:
#Load Additional files
tracks_per_year=pd.read_csv('data/AdditionalFiles/tracks_per_year.txt',delimiter='<SEP>',engine='python',header=None,index_col=1,names=['year','trackID','artist','title'])
unique_tracks=pd.read_csv('data/AdditionalFiles/unique_tracks.txt',delimiter='<SEP>',engine='python',header=None,index_col=0,names=['trackID','songID','artist','title'])
unique_artists=pd.read_csv('data/AdditionalFiles/unique_artists.txt',delimiter='<SEP>',engine='python',header=None,index_col=0,names=['artistID','artistMID','randomTrack','name'])
artist_location=pd.read_csv('data/AdditionalFiles/artist_location.txt',delimiter='<SEP>',engine='python',header=None,index_col=0,names=['artistID','lat','long','name','location'])

In [ ]:
#Check if indexes is unique and print the number of elements for each dataframe
print('Dataframe (Unique index, Number of elements)')
print('tracks_per_year ',(tracks_per_year.index.is_unique,tracks_per_year.shape[0]))
print('unique_tracks ',(unique_tracks.index.is_unique,unique_tracks.shape[0]))
print('unique_artists ',(unique_artists.index.is_unique,unique_artists.shape[0]))
print('artist_location ',(artist_location.index.is_unique,artist_location.shape[0]))

The covers dataset (SHS_testset.txt and SHS_trainset.txt) were organised in a very special way where group (named "cliques") list some tracks that are interrelated (music covers and original track). The function **read_shs_files** is used to import the files keeping the "clique" configuration.

In [ ]:
def read_shs_files(pathToFile):
    f = open(pathToFile)
    s = StringIO()
    cur_ID = None
    for ln in f:
        if not ln.strip():
                continue
        if ln.startswith('%'):
                cur_ID = ln.replace('\n','<SEP>',1)
                continue
        if cur_ID is None:
                print ('NO ID found')
                sys.exit(1)
        s.write(cur_ID + ln)
    s.seek(0)
    df = pd.read_csv(s,delimiter='<SEP>',engine='python',header=None,names=['shsID','trackID','artistID','shsPerf'])
    return df

In [ ]:
#Import the two SHS datasets and concatenate them
SHS_testset=read_shs_files('data/SHS_testset.txt')
SHS_trainset=read_shs_files('data/SHS_trainset.txt')
covers=pd.concat([SHS_testset,SHS_trainset])
covers.shsID=covers.shsID.str.strip('%')
covers.head()

### 2. Clique organisation (Multi-level indexing)
As said before, the cover dataset is organised as cliques that group some specific tracks that are interrelated. 
In order to keep this structure, we decide to use a multilevel index with cliques (need to transform shsID category in int) and then use the ranking according the released date of the track (year attribute) for the second index (thus, 0 will be the original song).

In [ ]:
#Convert shsID to clique id (first convert to category and get a code)
covers=covers.assign(clique_id=(covers.shsID.astype('category')).cat.codes)
#Remove the shsID column (useless since we have the clique_id now)
covers.drop('shsID',axis=1,inplace=True)

In order to use the informations contained in the metadata files first, we merged some necessary attributes (name of the artist, title of the track, released date) from the MSD dataframes named before.

In [ ]:
#Merge with unique_artists dataframe to find the artist name for each track (no taking consideration of featuring since we take only the name of the artist assigned with the track)
covers=covers.merge(unique_artists[['name']],how='left',left_on='artistID',right_index=True)
#Merge with unique_tracks dataframe to find the track name
covers=covers.merge(unique_tracks[['title']],how='left',left_on='trackID',right_index=True)
#Merge with tracks_per_year dataframe to find the year of each track
covers=covers.merge(tracks_per_year[['year']],how='left',left_on='trackID',right_index=True)

In [ ]:
covers.head()

Here is printed some useful informations about the cover dataset (the basis of our work) :

In [ ]:
print('Number of tracks :', covers.shape[0])
print('Number of cliques :', max(covers.index)+1) #Number of cliques (+1 because id starts at 0)
print('Number of unique tracks :', len(covers.trackID.unique())) 
print('Number of unique artists :', len(covers.artistID.unique()))
print('Number of missing trackID :', len(covers[covers.trackID.isnull()]))
print('Number of missing artistID :', len(covers[covers.artistID.isnull()]))
print('Number of missing years :', len(covers[covers.year.isnull()]))

In [ ]:
covers=covers.sort_values(['clique_id', 'year'], ascending=[True, True]).reset_index() #Reset index according clique_id and year
covers.drop('index',axis=1,inplace=True) #Drop the previous index

### 3. Addition of the language and the year of each track (SHS website web-scraping)
With the results printed above, we noticed that 4796 years were missing (26%) and since we thought to detect the original song by ranking the tracks year in each clique, we needed to find a way to get them.
Furthermore, year isn't necessarly sufficient informations to discriminate the tracks (cover appears sometimes in the same year than the original one), thus it will be better to have the released date for ALL the tracks if the information is available in the SHS website.

Indeed, the Second Hand Songs website allows API request on their database (limited to 1000 requests per hour).
Each track has a performance page where we can have access additional informations about the track as the language, the released date and the original song of the specific cover. In the SHS website, the performance id (that is used in the URL to access to the performance page) is available in our cover dataframe (shsPerf).
Nevertheless, we have negative values of shsPerf and it corresponds to missing values.

Thus, we have two ways to access extract the language/year/original song via web-scrapping :
- For valid SHS performance ID, access to the performance page (e.g. 'https://secondhandsongs.com/performance/1983') and web-scrapping of the Language and Released date informations using the perfInfo() function.
- For invalid SHS performance ID, API request to the search page (e.g. 'https://secondhandsongs.com/search/performance?title=blackbird&performer=beatles'), extract the perf ID with the find_PerfID() and then use the perfInfo() function.

In [ ]:
print('Number of missing years with valid shsPerf (API request on the performance page) :',len(covers[(covers.year.isnull()) & (covers.shsPerf != -1)]))
print('Number of missing years with invalid shsPerf (API request on the search page to find shsPerf) :',len(covers[(covers.year.isnull())])-len(covers[(covers.year.isnull()) & (covers.shsPerf != -1)]))

In [ ]:
pickle.dump(covers,open('covers.p','wb'))

In order to test these algorithms, we have worked for now with a part of the cover dataframe (part dataframe), containing only 962 tracks but being representative of the covers dataframe.

In [ ]:
covers.shape

In [ ]:
#Merge with the unique_tracks dataframe to get the name of the artist for the track (take featuring as well), it will be useful for the find_shsPerf function 
covers=covers.merge(unique_tracks[['artist']],how='left',left_on='trackID',right_index=True)
covers.head()

In [ ]:
print('Number of cliques in the subset :', len(covers.clique_id.unique()))
print('Number of tracks in the subset :', covers.shape[0])
print('Number of missing years in the subset :', len(covers[covers.year.isnull()]))
print('Number of invalid shsPerf in the subset :', len(covers[covers.shsPerf<0]))

In [ ]:
#API request to find the SHS perf for the unvalid ones (negative values)
def find_shsPerf(x):
    title=part.iloc[x]['title']
    artist=part.iloc[x]['artist']
    shsPerf=part.iloc[x]['shsPerf']
    
    if shsPerf<0:
        title=title.replace('.', '').replace('_', '').replace('/', '').lower().replace(' ','+')
        artist=artist.replace('.', '').replace('_', '').replace('/', '').lower().replace(' ','+')
        r=requests.get('https://secondhandsongs.com/search/performance?title='+title+'&op_title=contains&performer='+artist+'&op_performer=contains')
        soup = BeautifulSoup(r.text, 'html.parser')
        results=soup.find('tbody')

        if results is None :
            new_shsPerf=0
        else:
            new_shsPerf=int(results.find('a',attrs={'class':'link-performance'})['href'].split('/')[2])
    else :
        new_shsPerf=shsPerf
        
    return new_shsPerf

In [ ]:
#Find the shsPerf for the tracks which doesn't have valid ones (substract 2055 to part dataframe index to start with index=0)
#covers_withSHS=covers.shsPerf.index.map(lambda x: find_shsPerf(x-18196)) 

In [ ]:
#covers_withSHS=pd.concat([part1, part2, part3, part4, part5, part6])

In [ ]:
#pickle.dump(covers_withSHS,open('data/covers_withSHS.p','wb'))
covers_withSHS=pickle.load(open("data/covers_withSHS.p","rb"))

In [ ]:
covers_withSHS[covers_withSHS.shsPerf==0]

In [ ]:
covers_part=covers_withSHS[10000:18196]

We still can't access the SHS pages for **1050** music covers. We will need to decide if remove them since it will be impossible to find the missing release date and/or the language of the track.
For now, we will compute the perfInfo_SHS for all the dataset.

In [ ]:
#API request to SHS website for the page of a specific performance (defined as shsPerf) to extract Language and Date
def perfInfo_SHS(shsPerf):
    if shsPerf==0:
        perfLanguage='Unavailable'
        perfDate='Unavailable'
        original_shsPerf='Unavailable'
        
    else :
        r = requests.get('https://secondhandsongs.com/performance/'+str(shsPerf))
        soup = BeautifulSoup(r.text, 'html.parser')
        perfMeta=soup.find('dl',attrs={'class':'dl-horizontal'})
        if perfMeta is None:
            perfLanguage='Missing'
            perfDate='Missing'
            original_shsPerf='Missing'
        else :
            perfLanguage=perfMeta.find('dd',attrs={'itemprop':'inLanguage'})
            if perfLanguage is None :
                perfLanguage='Missing'
            else :
                perfLanguage=perfLanguage.text

            perfDate=perfMeta.find('div',attrs={'class':'media-body'})
            if perfDate is None :
                perfDate='Missing'
            else :
                perfDate=perfDate.find('p').text.split('\n')[2].strip(' ')

            original_shsPerf=soup.find('section',attrs={'class':'work-originals'})
            if original_shsPerf is None :
                original_shsPerf='Missing'
            else :
                original_shsPerf=original_shsPerf.find('a',attrs={'class':'link-work'})['href'].split('/')[2]

    return perfLanguage,perfDate,original_shsPerf

In [ ]:
#covers_part['language'], \
#covers_part['date'], \
#covers_part['original_shsPerf']= zip(*covers_part.shsPerf.map(perfInfo_SHS))

In [ ]:
#pickle.dump(covers_part,open('data/covers_10000_18196.p','wb'))

In [ ]:
covers_part1 = pickle.load(open("data/covers_0_10000.p","rb"))
covers_part2 = pickle.load(open("data/covers_10000_18196.p","rb"))

In [ ]:
covers=pd.concat([covers_part1, covers_part2])

In [ ]:
covers.shape

We noticed several problems with this intermediate result :

1/ The track year (year) is sometimes different form the released date (date) we've extracted from the SHS website, we will prefer the data information found in the SHS website.

2/ Some original performance that were found in the SHS website don't appear in the clique so need to add these tracks to our dataframe.
- See for each clique if there is at least one original song defined
- See if the original song is unique (in each clique)
- Check if we have information about the orginal song in our dataframe or if we need to web-scrap from SHS website

In [ ]:
#At least one original song in each clique ?
#Replace missing values by Nan in the original_shsPerf column (to don't count them as unique values)
covers.original_shsPerf.replace(['Missing','Unavailable'],np.NaN,inplace=True)
count_unique=covers.groupby('clique_id').original_shsPerf.nunique(dropna=True)

In [ ]:
#Is the original song unique in each clique ?
len(count_unique[count_unique>1])

In [ ]:
covers[covers.clique_id==271]

In [ ]:
#Number of cliques with no original song found
len(count_unique[count_unique==0])

In [ ]:
#
shsPerfUnique=pd.DataFrame(covers.shsPerf.unique())
originalPerfUnique=pd.DataFrame(covers.originalPerfUnique.unique())
merge=originalPerfUnique.merge(shsPerfUnique,how='left')
merge

In [ ]:
covers

Here is the information we obtained for now. We noticed several problems that needs to be fixed later :
- The track year (year) is sometimes different form the released date (date) we've extracted from the SHS website, we will prefer the data information found in the SHS website.
- Some languages and some dates are missing (we will consider droping these covers or restrict our analyse concerning these parameters to only a subset of cliques).
- Some original performance that were found in the SHS website don't appear in the clique so need to add these tracks to our dataframe (find them in the cluster or webscrap directly to the SHS website)
- We may still have a problem to find the ranking (just discriminate the original track and do not sort the music covers).

The work for this part will be to extend the analysis to the entire cover dataframe, resolve problems cited above and finish the multilevel indexing using ranking according the date in each clique. The API request is limited for the Second Hand Songs (SHS) website to 1000 requests per hour. Due to the large number of requests needed (668 to resolve the missing SHS problem and 18196 to find the Language/Year/Original Song), we'll maybe ask to the SHS team an exception to remove this limitation.

Then, the goal is to add to this dataframe the informations we've extracted through the cluster (tempo, song hotness), through the LastFM dataset (genre) and through the artist_location table (country of the artist) with the methods we describe in the next section. 

Thus, we'll have all the informations required to start the analysis of our music covers.

In [ ]:
#Take the clique id we defined as id of the dataframe (not unique index for now)
#covers.set_index('clique_id',inplace=True)
#covers.sort_index(inplace=True)

#Compute the order of songs for each clique
#covers['rank']=covers.groupby('clique_id')['year'].rank(method='dense',ascending=True).astype('int')

### 4. Access to files (tempo / dancability)

We open the first file of the subset, to check what the HDF5 keys are and then we read each of them.

In [ ]:
with pd.HDFStore("data/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5") as hdf:
    print(hdf.keys())

In [ ]:
pd.read_hdf("data/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5","/analysis/songs")

In [ ]:
pd.read_hdf("data/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5","/metadata/songs")

In [ ]:
pd.read_hdf("data/MillionSongSubset/data/A/A/A/TRAAAAW128F429D538.h5","/musicbrainz/songs")

We only need to extract <tt>tempo</tt> and <tt>song_hotttnesss</tt>, here is an example of how to do that on the subset :

In [ ]:
tempo = []
hotness = []

files = glob.glob("data/MillionSongSubset/data/A" + "/[A-Z]/[A-Z]/*")
for f in files:
    tempo.append(pd.read_hdf(f,"/analysis/songs")["tempo"][0])
    hotness.append(pd.read_hdf(f,"/metadata/songs")["song_hotttnesss"][0])

In [ ]:
tempo = np.asarray(tempo)
hotness = np.asarray(hotness)
print(tempo)
print(hotness)

In [ ]:
print("Number of tracks =", len(files))
print("with missing hotness values =", np.sum(np.isnan(hotness)))

We already got 3268 unknown hotness values and we only tested on a subset of 7620 song, so we can expect to have that information for only a little over half of our final dataset. Maybe we won't use it.

Once all the files are accessible on the cluster, we will have to go through our SHS dataset and get those attributes for each track_id.
We will do so in the following way : (the paths are just examples on the subset)

In [ ]:
tempo = []
hotness = []

my_file = Path("/path/to/file")
for track in covers["trackID"]:
    folder1 = track[2]
    folder2 = track[3]
    folder3 = track[4]
    folder_path = "data/MillionSongSubset/data/" + folder1 + "/" + folder2 + "/" + folder3 + "/"
    track_path = folder_path + track + ".h5"
    if Path(track_path).exists(): #to delete later
        tempo.append(pd.read_hdf(track_path,"/analysis/songs")["tempo"][0])
        hotness.append(pd.read_hdf(track_path,"/metadata/songs")["song_hotttnesss"][0])

In [ ]:
print(len(tempo))
print(np.sum(~np.isnan(hotness)))

Unsurprisingly, we only found 204 of those tracks in the subset and 128 of them have a hotness value.

### 5. Determine artist location for spatial analysis

In [ ]:
#Load Additional files
#unique_artists=pd.read_csv('data/AdditionalFiles/unique_artists.txt',delimiter='<SEP>',engine='python',header=None,index_col=0,names=['artistID','artistMID','randomTrack','name'])
unique_artists=pd.read_csv('data/AdditionalFiles/unique_artists.txt',delimiter='<SEP>',engine='python',header=None,index_col=0,names=['artistID','artistMID','randomTrack','name'])
artist_location=pd.read_csv('data/AdditionalFiles/artist_location.txt',delimiter='<SEP>',engine='python',header=None,index_col=0,names=['artistID','lat','long','name','location'])
artist_location.head()

We now load a subset of Second Hand Song

In [ ]:
def read_shs_files(pathToFile):
    f = open(pathToFile)
    s = StringIO()
    cur_ID = None
    for ln in f:
        if not ln.strip():
                continue
        if ln.startswith('%'):
                cur_ID = ln.replace('\n','<SEP>',1)
                continue
        if cur_ID is None:
                print ('NO ID found')
                sys.exit(1)
        s.write(cur_ID + ln)
    s.seek(0)
    df = pd.read_csv(s,delimiter='<SEP>',engine='python',header=None,names=['shsID','trackID','artistID','shsPerf'])
    return df[['trackID', 'artistID', 'shsPerf']]

We retrieve the artists' names using the unique_artists.txt file and we assign a location for each track using the artist_location.txt.

In [ ]:
def get_location(x) : 
    if x in artist_location.index:
        return artist_location.get_value(x, 'location')
    else : 
        return np.nan
    
data=read_shs_files('data/SHS_testset.txt')
data['artist'] = data['artistID'].map(lambda x : unique_artists.get_value(x, 'name'))
data['location'] = data['artistID'].map(lambda x : get_location(x))
data.head()

We now create the function finding the country for each location. In order to do that we wille use three different python packages : pycountry, us, and geopy, as geopy.geocoders does not support too much requests. 

- First, we will use the pycountry package to extract countries if location contains one. 


- If we didn't match any country in pycountry, we will use the us package to check if a us state is present in the location. From the data, we have observed that if the location refer to a us state, the location is either only defined by the state, or the state is the last element of the location.


- If the two precedent methods does not succeed, we will use the geopy.geocoders package, using Nominatim( ).


- We will manually define countries for some location as they are sometimes mispelled, troncated or refer to a website link.

In [ ]:
geolocator = Nominatim()

def get_country(x):
    if x == np.nan:
        return x
    x = x.replace("-", ",")
    for c in pycountry.countries:
        if "England" in x or "UK" in x: 
            return "United Kingdom"
        elif c.name.lower() in x.lower():
            return c.name
    refactorlast = x.split(",")[-1].replace(" ", "")
    refactorfirst = x.split(",")[0]
    usstatelast = us.states.lookup(refactorlast)
    usstatefirst = us.states.lookup(refactorfirst)
    if usstatelast != None or usstatefirst != None:
        return "United State of America"
    elif x == "Swingtown":
        return "United State of America"
    elif x == "<a href=\"http://billyidol.net\" onmousedown='UntrustedLink.bootstrap($(this), \"fc44f8f60d13ab68c56b3c6709c6d670\", event)' target=\"_blank\" rel=\"nofollow\">http://billyidol.net</a>":
        return "United Kingdom"
    elif x == "Lennox Castle, Glasgow" or x == "Knowle West, Bristol, Avon, Engla"\
        or x == "Goldsmith's College, Lewisham, Lo" or x == "Julian Lennon&#039;s Official Facebook Music Page"\
        or x == "Sydney, Moscow, Pressburg" or x == "Penarth, Wales to Los Angeles" or x == "Leicester, Leicestershire, Englan":
        return "United Kingdom"
    elif x == "Vancouver, British Columbia, Cana":
        return "Canada"
    elif x == "Washington DC" or x == "Philladelphia" or "New Jersey" in x:
        return "United State of America"
    elif "Czechoslovakia" in x :
        return "Česko"
    elif x == "Jaded Heart Town":
        return "Germany"
    elif x == "RU" or x == "Russia":
        return "Russia"
    else :
        location = geolocator.geocode(x, timeout=None)
        return location.address.split(",")[-1]

In [ ]:
#data['country'] = data['location'].map(lambda x : get_country(x))

The only problem with geopy is that it returns a country in its native language. To uniform our data, we create a function that translates manually the countries in English.

In [ ]:
def rename(x):
    if "België - Belgique - Belgien" in x:
        return "Belgium"
    elif "Brasil" in x:
        return "Brazil"
    elif "United State" in x:
        return "United States of America"
    elif "Italia" in x:
        return "Italy"
    elif "Norge" in x:
        return "Norway"
    elif "España" in x:
        return "Spain"
    elif "Nederland" in x :
        return "Netherlands"
    elif "Suomi" in x :
        return "Finland"
    elif "Sverige" in x :
        return "Sweden"
    elif "UK" in x :
        return "United Kingdom"
    elif x[0] == " ":
        return x[1:]
    else : 
        return x

In [ ]:
#data['country'] = data['country'].map(lambda x : rename(x))
#pickle.dump(data, open( "data.p", "wb" ) )
data_country = pickle.load(open("data/data_country.p", "rb"))

In [ ]:
data_country.head()

### 6. Addition of the genre for each track (Use of LastFM dataset and external website for genre listing)

To find the genre of a song, we will use the LastFM dataset that contains a list a tags for each song.
Since the dataset is from the MillionSongDataset, we will not use all of the available tracks from LastFM but, but only the ones contained in the SecondHandSong dataset.

In [ ]:
# Loading the files if they are in the SecondHandSong dataset and create the dataframe
covers_df = pickle.load(open("data/covers.p","rb"))
list_tracks = covers_df.trackID
test_path = "../../lastfm_test"
train_path = "../../lastfm_train"

genre_df = pd.DataFrame()
def create_dataFrame(genre_df):
    for track in list_tracks:
        folder1 = track[2]
        folder2 = track[3]
        folder3 = track[4]
        folder_path = "/" + folder1 + "/" + folder2 + "/" + folder3 + "/"
        track_path = folder_path + track + ".json"
        if glob.glob(train_path + track_path) != []:
                genre_df = genre_df.append(pd.DataFrame.from_dict(json.load(open(train_path + track_path)), orient="index").transpose())
        elif glob.glob(test_path + folder_path + track) != []:
                genre_df = genre_df.append(pd.DataFrame.from_dict(json.load(open(test_path + track_path)), orient="index").transpose())
    genre_df = genre_df.reset_index()
    return genre_df

#tracks_with_tags = create_dataFrame(genre_df)
tracks_with_tags = pickle.load(open("tracks_with_tags", "rb"))

We now list the unique tags in the resulting dataframe. Due to a time limit for the computation of the matching, we will first test on a subset.

In [ ]:
tags = list()
for i in range (0,1000):
    tags = tags + tracks_with_tags.tags[i]
    
tags = np.unique(tags).tolist()

A lot of tags contains useless information, thus we first proceed to a pre-cleaning.

In [ ]:
clean_tags = {}
def clean_tag(x):
    clean = x.replace("ooo", "")
    clean = clean.replace("-o", "")
    clean = clean.replace("o-", "")
    clean = clean.replace("- ", "")
    clean = clean.replace("-", "")
    clean_tags[x] = clean
for t in tags:
    clean_tag(t)

In order assign a genre to each song, we will use their different tags and try to match it with a list of genre obtained by webscrapping the http://www.musicgenreslist.com website. For more details on the webscrapping see the notebook Genre Webscrapping.ipynb.

In [ ]:
map_genres = pickle.load(open("data/map_genres", "rb"))
all_genres = pickle.load(open("data/all_genres.p", "rb"))

We then use the Sequence Matcher package to match tags to the web-scrapped genres.

In [ ]:
threshold = 0.80
def match_genres():
    i = 0
    genre_map = {}
    no_match = list()
    for ind in range(0,len(tags)):
        name1 = tags[ind]
        if i%1000 == 0:
            print(i)
        if clean_tags[name1] == "":
            genre_map[name1] = np.nan
        best_ratio = 0
        match = ""
        for name2 in map_genres.keys():
            if name2.lower() in name1.lower():
                for subgenre in map_genres[name2]:
                    ratio = SequenceMatcher(None,name1.lower(),name2.lower()).ratio()
                    if ratio > best_ratio:       # we find the maximum similarity
                        best_ratio = ratio
                        match = name2
                if (best_ratio > threshold):     # if it's superior to our threshold we add that couple to the mapping
                    genre_map[name1] = match
                else:
                    genre_map[name1] = name2
        if match == "":
            for subgenre in all_genres:
                ratio = SequenceMatcher(None,name1.lower(),name2.lower()).ratio()
                if ratio > best_ratio:       # we find the maximum similarity
                    best_ratio = ratio
                    match = name2
            if (best_ratio > threshold):     # if it's superior to our threshold we add that couple to the mapping
                genre_map[name1] = match
            else :
                genre_map[name1] = np.nan
        i = i+1
    return (genre_map, no_match)